# Installation


In [ ]:
%%capture
import os, re, random
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

# Dataset


In [ ]:
from datasets import load_dataset

dataset = load_dataset("INK-USC/riddle_sense", split="train", trust_remote_code=True)
ansKey = {'A':0,'B':1,'C':2,'D':3,'E':4}

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
print(dataset['question'][0])
print(dataset['answerKey'][0])
print(dataset['choices'][0]['text'][4])

A man is incarcerated in prison, and as his punishment he has to carry a one tonne bag of sand backwards and forwards across a field the size of a football pitch.  What is the one thing he can put in it to make it lighter?
E
hole


In [ ]:
def getExample():
  rndm = random.randint(501, 3510)
  shot = "Riddle: " + dataset['question'][rndm] + "\n"
  ans = dataset['answerKey'][rndm]
  shot += "Answer: " + dataset['choices'][rndm]['text'][ansKey[ans]]
  return shot

print(getExample())

Riddle: 1 man runs 3 blocks and then comes home and there is 2 maced men who r they?
Answer: catcher and umpier


# Model

In [ ]:
#code from https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_v0.3_(7B)-Conversational.ipynb

from unsloth import FastLanguageModel
import torch
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3",
    max_seq_length = 4096, # Choose any!
    dtype = None, # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = True, # Use 4bit quantization to reduce memory usage. Can be False.
)
EOS_TOKEN = tokenizer.eos_token

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Could not import trl.trainer.alignprop_trainer: Failed to import trl.trainer.alignprop_trainer because of the following error (look up to see its traceback):
Failed to import trl.models.modeling_sd_base because of the following error (look up to see its traceback):
Failed to import diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion because of the following error (look up to see its traceback):
Failed to import diffusers.loaders.ip_adapter because of the following error (look up to see its traceback):
JITCallable._set_src() takes 1 positional argument but 2 were given
Unsloth: Could not import trl.trainer.ddpo_trainer: Failed to import trl.trainer.ddpo_trainer because of the following error (look up to see its traceback):
Failed to import trl.models.modeling_sd_base because of the following error (look up to see its traceback):
Fa

==((====))==  Unsloth 2025.12.4: Fast Mistral patching. Transformers: 4.55.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
%%capture
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "mistral",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

#FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# FINETUNING

In [ ]:
traindataset = load_dataset("INK-USC/riddle_sense", split="train[:500]", trust_remote_code=True)
traindataset = traindataset.map(lambda example, idx: {"question": 'Riddle: '+ example["question"] + '\nAnswer: ' + example["choices"]['text'][ansKey[example['answerKey']]] + EOS_TOKEN}, with_indices=True)
traindataset = traindataset.map(remove_columns=['choices','answerKey'])
print(traindataset['question'][0])

Riddle: A man is incarcerated in prison, and as his punishment he has to carry a one tonne bag of sand backwards and forwards across a field the size of a football pitch.  What is the one thing he can put in it to make it lighter?
Answer: hole</s>


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",

                      "embed_tokens", "lm_head",], # Add for continual pretraining
    lora_alpha = 32,
    lora_dropout = 0.01, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.01.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:1222: UserWarning: Model has `tie_word_embeddings=True` and a tied layer is part of the adapter, but `ensure_weight_tying` is not set to True. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. Check the discussion here: https://github.com/huggingface/peft/issues/2777
  warnings.warn(msg)
Unsloth 2025.12.4 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import UnslothTrainer, UnslothTrainingArguments

def formatting_prompts_func(examples):
    instructions = examples["question"]
    texts = []
    for instruction in instructions:
        # The 'question' field already contains the formatted riddle and answer
        text = f"{instruction}"
        texts.append(text)
    return texts # Changed to return the list directly

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = traindataset,
    #ataset_text_field = "question", # Remove this line as formatting_func will handle it
    max_seq_length = 4096,
    dataset_num_proc = 8,
    formatting_func = formatting_prompts_func,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        gradient_checkpointing=True,

        warmup_ratio = 0.1,
        num_train_epochs = 1,

        learning_rate = 5e-5,
        embedding_learning_rate = 5e-6,

        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.00,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/500 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 32
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 603,979,776 of 7,852,003,328 (7.69% trained)


Step,Training Loss
1,3.374700
2,3.816800
3,3.008500
4,2.959500
5,2.277200
6,2.639300
7,2.206400
8,2.283600
9,2.314800
10,2.086200


Function to prompt the model

In [ ]:
def promptMinstral(myprompt):
  messages = [{"from":"human", "value": myprompt}]
  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize = True,
      add_generation_prompt = True, # Must add for generation
      return_tensors = "pt",
  ).to("cuda")
  outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True)
  return tokenizer.batch_decode(outputs)[0]

# Regex

In [ ]:
import re
import math

def decodeOutput(riddle):
  decoded = re.split(r"\[/INST\]",riddle)
  decoded = decoded[1] # cuts off prompt

  riddle = re.search("[r|R]iddle: (.*)(<)+?", decoded)
  if not riddle:
    riddlesplit = decoded.split('\n')
    riddle = riddlesplit[0] if len(riddlesplit[0]) > 1 else riddlesplit[1]
    if '<' in riddle:
      riddle = riddle.split('<')[0]
  else:
    riddle = riddle[1]

  answer = re.search("[a|A]nswer: (.*)(<)+?", decoded)
  if not answer:
    answer = "None"
  else:
    answer = answer[1]

  if answer == "None" and not riddle:
    print(decoded)
  return riddle, answer

In [ ]:
#myInput = """Your job is to generate a riddle given another riddle. Only send the riddle and answer. Following is an example of a riddle you should emulate. Example: """ + getExample()
#myInput = """You're a specialist in creating riddles. You always format the riddle following the example, putting the riddle and answer on one line each:\"""" + getExample() + "\"\nNow its your turn. Create a riddle."
myInput = """Your job is to generate a riddle given another riddle. Only send the riddle and answer. Following is an example of a riddle you should emulate. Example: \n""" + getExample()

rawOutput = promptMinstral(myInput)

print(rawOutput)
print("---------------------------\n")

riddle, answer = decodeOutput(rawOutput)
print(riddle, answer)

<s>[INST] Your job is to generate a riddle given another riddle. Only send the riddle and answer. Following is an example of a riddle you should emulate. Example: 
Riddle: you ride into a town on wednesday,you stay three days and leave on wednesday.   how is that possible?
Answer: your horses name is wednesday [/INST]: you ride into a town on wednesday,you stay three days and leave on wednesday.   how is that possible?
Answer: your horses name is wednesday</s>
---------------------------

: you ride into a town on wednesday,you stay three days and leave on wednesday.   how is that possible? your horses name is wednesday


Location nodes and map

In [1]:
class Location:
  def __init__(self, block = False):
    self.blocked = block
    # room connections
    self.north = None
    self.east = None
    self.west = None
    self.south = None
    # list of items - there's one case where there can be more than one item but its useful
    self.items = []
    # description of the room
    self.desc = None
    # bool as to whether a riddle can occur in the room
    self.riddle = False
    # message that the parser reads if the way is blocked.
    self.blocked_msg = None

In [2]:
# Basicest map
start = Location()
hallway = Location()
candleRoom = Location()
bugRoom = Location(True)
escapeRoom = Location()

# Joining up the locations
start.east = hallway

hallway.east = candleRoom
hallway.west = start
hallway.south = bugRoom

candleRoom.west = hallway

bugRoom.north = hallway
bugRoom.south = escapeRoom

escapeRoom.north = bugRoom

# Set items in the rooms
candleRoom.items.append("candle")
hallway.items.append("rocks")

# Establishes the rooms the riddles will occur in
start.riddle = True
candleRoom.riddle = True
escapeRoom.riddle = True

# Set descriptions for the room
start.desc = """A dim room in the stone temple. You woke up here. It smells like mold and is too dark to make much out.
You're pretty sure you've seen Indiana Jones in a place like this.
Only he had a gun."""

hallway.desc = """A room that connects to three others.
This had to be the ancient equivalent of a hallway?
There was a fourth door at one point but all that stands in its place is a pile of rubble.
The room to East looks surprisisngly bright.
Echoes carry in from the South room."""

candleRoom.desc = """A surprisingly bright room. It takes your eyes some time to adjust.
Candles line the walls, still lit despite nobody living here.
You almost forget how hot it is until you feel a bead of sweat rolling down your face."""

bugRoom.desc = """The echo you heard grows louder and louder until you're face to face with its source.
Bugs. Cockroaches, centipedes, spiders. They scuttle about the darkness.
You can't even see the tiles you're standing on.
Just past the buzzing you can see light - daylight even."""

escapeRoom.desc = """You can just barely see the daylight! You're nearly freed.
'escape' to free yourself!
"""

# Set block messages to the room
bugRoom.blocked_msg = "You take a few steps into the bug infested room only for something to sink its teeth into you. Then another bite. You quickly run back,"
escapeRoom.blocked_msg = "You cannot leave. The Sphinx won't let you."

Parser class

In [13]:
class Parser:
  def __init__(self, debug = False):
    self.lives = 3
    self.location = start
    self.inventory = []
    self.level = 0
    self.victory = False
    self.debug = debug
    self.explanation = None

  def comp(self, response, answer):
      # answer - compares the response given to the answer the model provided.
      #
      # inputs: user response and the answer given by the llm
      # returns: True or False
      # True - the answer is correct, it passes the threshold
      # False - the answer is incorrect.


      # tokenize
      resp_tokens = response.lower().split()
      ans_tokens = answer.lower().split()

      # build frequency dicts
      resp_vec = {}
      for w in resp_tokens:
        resp_vec[w] = resp_vec.get(w, 0) + 1

      ans_vec = {}
      for w in ans_tokens:
        ans_vec[w] = ans_vec.get(w, 0) + 1

      # vocabulary
      all_words = set(resp_vec.keys()) | set(ans_vec.keys())

      # numeric vectors
      v1 = [resp_vec.get(w, 0) for w in all_words]
      v2 = [ans_vec.get(w, 0) for w in all_words]

      # cosine similarity
      dot = sum(a * b for a, b in zip(v1, v2))
      mag1 = math.sqrt(sum(a * a for a in v1))
      mag2 = math.sqrt(sum(b * b for b in v2))

      cosine_sim = 0 if (mag1 == 0 or mag2 == 0) else dot / (mag1 * mag2)

      # threshold check
      return True if cosine_sim >= 0.4 else False

      return True if eval >= 0.4 else False # thresholds the % match of the

  def ask(self):
    # promtpts the LLM to output a new riddle in the form of a json file
    # that includes the riddle, a hint and the answer. The function runs a nested loop that
    # checks users answer to give you multiple tries.
    #
    #
    # inputs : None
    # output : True if the question was correctly answered, False if not.

    # Input prompt
    myInput = """Your job is to generate a riddle given another riddle. Only send the riddle and answer. Following is an example of a riddle you should emulate. Example: """ + getExample()
    minimenu = """You may 'give up' and get another question\nSometimes the Sphinx is stupid. Say 'what' when its stupid.\nYour answer: """


    # Generate text
    modelOutput = promptMinstral(myInput)
    self.explanation = modelOutput

    # Decode and print the output
    gameriddle, gameanswer = decodeOutput(modelOutput)

    riddle_loop = True
    while riddle_loop and self.lives > 0: # different from the gameloop, riddle loop, consists only of the sphinx's question and answering.
      if self.level == 0:
        print("The sphinx lunges forward, ready to pounce on you. You see your life flash before your eyes before it asks,\n\""+ gameriddle + "\"")
      if self.level == 1:
        print("You're unsure of when the sphinx cornered you but you know what comes next. It bares its fangs this time and snarls out,\n\""+ gameriddle + "\"")
      if self.level == 0:
        print("You've never seen eyes with such murderous intent. The beast wants you dead. And you want out. End this here.\n\""+ gameriddle + "\"")
      if self.debug:
        print("======================================================")
        print("BACKEND DETAILS: ")
        print(modelOutput)
        print(gameriddle)
        print(gameanswer)
        print("======================================================")

      response = input(minimenu)
      response = response.lower()
      responseList = response.split(" ")


      if gameanswer == "None":
        print("The sphinx has a stroke and forgot to think of an answer to its own riddle. It's your lucky day")
        self.location.riddle = False
        return True
      if responseList[0] == 'give' and responseList[1] == 'up':
        print("Try as you might, you can't figure this one out. The sphinx claws at you for your incompetence, costing you dearly.")
        print("The answer the sphinx wanted was: " + gameanswer)
        self.lives -= 1
        return False
      elif responseList[0] == 'what':
        print("The sphinx's old age is clearly getting to it. You fail to understand the question and it gets really embarassed\nLike she's blushing and everything.\nIt leaves you for now...")
        print("The answer the sphinx wanted was: " + gameanswer)
        self.location.riddle = False
        return True
      if self.comp(response, gameanswer):
        print("That was correct.")
        print("The sphinx looks annoyed but it sulks off in defeat.")
        self.location.riddle = False
        print("From nowhere " + gameanswer + "drops to the floor from above. You can't see where it came from but it looks useful")
        self.location.items.append(gameanswer)
        return True
      else:
        print("The sphinx makes a noise like laughter. Then it swipes at you! Her claws cut your skin, leaving a nasty gash.\nYou were wrong. One step closer to being sealed inside forever.")
        self.lives -= 1

    # case where you failed the riddle and died.
    print("You try for an answer but it's too late. You've ran out of guesses. The beast lunges at you and sinks its teeth in. In an instant you're devoured.")
    print("The answer the sphinx wanted was: " + gameanswer)
    return False

  def prologue(self):
    # Prologue -
    # returns nothing but easier to navigate than multiple strings

    # Updates things that should change during the game
    self.lives = 3
    self.location = start
    self.inventory = []
    self.level = 0
    self.victory = False
    bugRoom.desc =  """The echo you heard grows louder and louder until you're face to face with its source.
Bugs. Cockroaches, centipedes, spiders. They scuttle about the darkness.
You can't even see the tiles you're standing on.
Just past the buzzing you can see light - daylight even."""

    start.riddle = True
    candleRoom.riddle = True
    escapeRoom.riddle = True
    bugRoom.blocked = True
    escapeRoom.blockked = True
    candleRoom.items = ["candle"]
    hallway.items =["rocks"]


    print("""\n▀█▀ █▀█ █ ▄▀█ █░░   █▀█ █▀▀   ▀█▀ █░█ █▀▀   █▀ █▀█ █░█ █ █▄░█ ▀▄▀
░█░ █▀▄ █ █▀█ █▄▄   █▄█ █▀░   ░█░ █▀█ ██▄   ▄█ █▀▀ █▀█ █ █░▀█ █░█

You were out on an expedition. All it was supposed to be was a quick trip to
a nearby 'temple.' Well, it was more of a tourist trap. You were pretty sure
anyway. The photos you'd seen looked plastic and from there wasn't a thing in
sight that couldn't have been bought off Amazon. You climbed stone stairs
through a forest to get here -

Then you slipped.

You wake up in a completely unfamilliar room. Your cheek was red, imprinted with
the stone flooring you'd been up against for the last several hours. The whole
room dark, creepy - and worst of all real.

"To leave," you hear a voice start. "You must first answer my Riddles Three."
Out from the darkness a sphinx slinks forward. Not the stone faced sculpture
you've seen in photos but a real sphinx. Its a menacing beast with the body
of a lion, wings and the face of an old lady.

\"You're a fool for coming here\" She snarls, \"and I'm never letting you leave.\"\n""")

  def game(self):
    # Game - Provides the game loop and main functionality of the game.
    # The game ends when the player quits, runs out of lives or they
    # correctly respond to three riddles.

    playing = True

    self.prologue()

    while(playing and self.lives > 0 and not self.victory): # GAMELOOP

      # Room message.
      print(self.location.desc+'/n')
      if(self.location.blocked):
        print("You aren't be able to go any further south at this time.")
      if self.location.riddle:
        print("Out from the shadows, the sphinx moves in front of the door. You must 'play' her game to pass.")

      # Takes user response and splits it before going through commands
      response = input("")
      response = response.lower()
      responseList = response.split(" ")

      # the big switch case controlling our game.
      if(responseList[0] == "q" or responseList[0] == "quit"):
        playing = False # User ends the game on their own, it ends here

      # plays - enters the riddle.
      elif(responseList[0] == 'play'):
        if self.location.riddle:
          if self.ask():
            self.level+=1
            print("Your chest swells with pride over your victory.")
          else:
            print("You failed the sphinx's riddle!")
        else:
          print("The sphinx is not in this room.")

      # Directional controls for the location system.
      elif(responseList[0] == "n" or responseList[0] == "north"):
          if self.location.north:
            if not self.location.riddle or self.debug:
                self.location = self.location.north
            else:
              print("The sphinx has you trapped in this room until you can answer her riddle!")
          else: print("There is no north exit.")
      elif(responseList[0] == "e" or responseList[0] == "east"):
          if self.location.east:
            if not self.location.riddle or self.debug:
              self.location = self.location.east
            else:
              print("The sphinx has you trapped in this room until you can answer her riddle!")
          else: print("There is no east exit.")
      elif(responseList[0] == "w" or responseList[0] == "west"):
          if self.location.west:
            if not self.location.riddle or self.debug:
              self.location = self.location.west
            else:
              print("The sphinx has you trapped in this room until you can answer her riddle!")
          else: print("There is no west exit.")
      elif(responseList[0] == "s" or responseList[0] == "south"):
        if self.location.south:
          if not self.location.blocked: # Progress will only ever go south, so we only need to check if its blocked here
              if not self.location.riddle or self.debug: # checks if in debug mode or if you've finished the riddle
               self.location = self.location.south
              else:
                print("The sphinx has you trapped in this room until you can answer her riddle!")
          else:
            print(self.location.blocked_msg)
        else: print("There is no south exit.")

      # give up command - going to phase out as we'll move the riddles to the ask function
      elif(responseList[0] == "give" and responseList[1] == "up"):
         print("You succumb to despair, losing a life")
         self.lives -= 1

      # why - why'd you get that last answer right?
      elif(responseList[0] == "why"):
        if self.explanation:
          print(self.explanation)
        else:
          print("Why what?")

      # look around - checks all exits
      elif(responseList[0] == "look"):
        if self.location.north:
          print("There is an exit to the north")
        if self.location.east:
          print("There is an exit to the east")
        if self.location.west:
          print("There is an exit to the west")
        if self.location.south:
          print("There is an exit to the south")
        if(self.location.blocked):
          print("You are won't be able to go south. Something is blocking the way. Probably bugs. Maybe scare them off with something hot?")
        if self.location.riddle:
          print("The sphinx is in this room, stopping you from leaving until you best it at its own game. Which is stupid. You wish you could play your own game instead.")
        if (self.location.items): # If there are items they're listed
          print("You see something useful: ")
          for item in self.location.items:
            print("\t a " + item)

      # get - it allows players to
      elif(responseList[0] == "get"):
        if self.location.items:
          print("You pickup the following items: " + str(self.location.items))
          self.inventory = self.inventory + self.location.items # conjoins the lists
          self.location.items = [] # empties out the list of items
        else:
          print("Have you gone mad already? There's nothing there.")

      # light - our main text based puzzle - players light up their candle to unblock the bug room
      elif(responseList[0] == "light" or "fire" in responseList):
        if "candle" or "wood" in self.inventory:
          if self.location == bugRoom:
            if "rocks" in self.inventory:
              print("You channel your inner caveman and start a fire. The bugs scurry away, leaving you free to head south in peace.")
              bugRoom.blocked = False
              bugRoom.desc = """Now that those pesky insects are gone you're free to roam the room. It smells like spilled ice cream. Maybe that's why there were all those bugs?\nThe room to the north is the hallway. Why head back though?\nTo the south is your escape. Now closer than ever"""
              self.inventory.remove("candle")
              self.inventory.remove("rocks")
            else:
              print("You rub your hands together to no avail. Unfortunately starting a fire takes some type of lighter... Drat, you left that at home.")
          else:
            print("There's no reason to do that here.")
        else:
          print("Despite your attempts to start a fire with your mind you have nothing to set ablaze")
      # escape - Win the game if sphinx doesn't have you in for a riddle
      elif(responseList[0] == "escape"):
        if self.location==escapeRoom and not self.location.riddle:
          print("You bust through a nearby wall into the open air! You've defeated the sphinx and now you're free!")
          if len(self.inventory) > 0:
            print("Along the way you got some loot!")
            for item in self.inventory:
              print("\t You won a" + item)
          self.victory = True
        if self.debug:
          print("You win through your amazing cheating skills")
          self.victory = True
      elif(responseList[0] == "inventory"):
        print("You look over the items you somehow fit in your pocket. You've got:")
        if len(self.inventory) > 0:
          for item in self.inventory:
              print("\t You won a" + item)

      # help gives the controls
      elif(responseList[0] == "help" or responseList[0] == "h"):
        print("Controls :")
        print("\t" +"play - In rooms where the sphinx lurks you can play its game. It won't let you go until you answer correctly.")
        print("\t" +"why - after being asked a riddle and answering successfully you can hear the sphinx's explanation of the riddle.")
        print("\t" + "q/quit - exits out of the game")

        print("\t" +"Directions: ")
        print("\t" + "\t" + "n/north - moves to the room north of this one")
        print("\t" + "\t" + "e/east - moves to the room east of this one")
        print("\t" + "\t" + "s/south - moves to the room south of this one")
        print("\t" + "\t" + "w/west - moves to the room west of this one")

        print("\t" +"look - describes the room's exits, blockages and any items in it. Very useful")
        print("\t" +"inventory - shows you the items you've obtained")
        print("\t" +"get - adds all items in a room to your inventory.")
        print("\t" +"There are more commands related to any items you might find! Try things out!")

      # else - the player misinput
      else:
        print("Not a registered command. Please type 'help' for a list of all controls.")


    # The gameloop has ended.
    if self.victory:
      print("""Congratulations! You're free from the sphinx's game and did not meet a fate most gruesome.
      You spend the next several hours trying to remember where you parked.
      Thanks for playing!
      █░█░█ █ █▄░█ █▄░█ █▀▀ █▀█
      ▀▄▀▄▀ █ █░▀█ █░▀█ ██▄ █▀▄
""")
    else:
      if self.lives >= 0:
        print("Your adventure ends here. Your heart comes to a sudden stop.")
      print("The tomb seals shut, dooming you to spend the rest of your short life within this miserable pit.")
      print("""█░█░█ █▀█ █▀▄▀█ █▀█   █░█░█ █▀█ █▀▄▀█ █▀█
▀▄▀▄▀ █▄█ █░▀░█ █▀▀   ▀▄▀▄▀ █▄█ █░▀░█ █▀▀""")

    return


In [14]:
pharohs_game = Parser(True)
pharohs_game.game()


▀█▀ █▀█ █ ▄▀█ █░░   █▀█ █▀▀   ▀█▀ █░█ █▀▀   █▀ █▀█ █░█ █ █▄░█ ▀▄▀
░█░ █▀▄ █ █▀█ █▄▄   █▄█ █▀░   ░█░ █▀█ ██▄   ▄█ █▀▀ █▀█ █ █░▀█ █░█

You were out on an expedition. All it was supposed to be was a quick trip to
a nearby 'temple.' Well, it was more of a tourist trap. You were pretty sure
anyway. The photos you'd seen looked plastic and from there wasn't a thing in
sight that couldn't have been bought off Amazon. You climbed stone stairs
through a forest to get here -

Then you slipped.

You wake up in a completely unfamilliar room. Your cheek was red, imprinted with
the stone flooring you'd been up against for the last several hours. The whole
room dark, creepy - and worst of all real.

"To leave," you hear a voice start. "You must first answer my Riddles Three."
Out from the darkness a sphinx slinks forward. Not the stone faced sculpture
you've seen in photos but a real sphinx. Its a menacing beast with the body
of a lion, wings and the face of an old lady.

"You're a fool for coming h

KeyboardInterrupt: Interrupted by user